<a href="https://colab.research.google.com/github/YG15/DataHack2019/blob/master/yonathan_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd
import numpy as np
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# !wget https://raw.githubusercontent.com/YG15/DataHack2019/master/src/functions/get_network_hierarchy.py
# from get_network_hierarchy import *

In [0]:
def diff_list(li1, li2): 
    return (list(set(li1) - set(li2))) 

def get_categ_col_dummy (df):    
    numerical_columns = df.describe().columns
    categorical_cols=diff_list(df.columns,numerical_columns)
    
    categorial = df[categorical_cols]
    #to prevent case-sensitive separation to different categories:
    categorial=categorial.apply(lambda x: x.str.lower(), axis=1)
    categorial_dummies = pd.get_dummies(categorial)
    df.drop(columns=categorical_cols, inplace=True)
    df=df.join(categorial_dummies)
    return df

In [55]:
url = 'https://armis-datahack.s3.amazonaws.com/all_devices.csv'
devices_df = pd.read_csv(url, index_col = 0)
devices_df['real_device'] = [float(str(id)+'.'+str(net)) for id,net in zip (devices_df.device_id,devices_df.network_id)]
devices_df.head()

,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version,real_device
113,0,1028623,MOBILE_PHONE,Galaxy S8,Samsung,Android,9,1028623.0
587,0,48047,MOBILE_PHONE,Galaxy Note 8,Samsung,Android,9,48047.0
668,0,123568,MOBILE_PHONE,H918,LG Electronics,Android,8.0.0,123568.0
830,0,95366,MOBILE_PHONE,iPhone 6,"Apple, Inc.",iOS,NaN,95366.0
886,0,1755023,TABLET,iPad,Apple,iOS,NaN,1755023.0


In [56]:
keep_os = ['Android', 'iOS', 'Mac OS X', 'Windows', 'watchOS',
       'Watch OS', 'Tizen', 'Symbian', 'Miui OS', 'Linux',
       'Axis Firmware', 'Firefox OS', 'Cisco IOS-XE', 'Cisco IOS',
       'Debian', 'Link-OS', 'Cisco NX-OS']
devices_df['operating_system'] = [ val if val in keep_os else 'OTHER' for val in devices_df.operating_system]
devices_df.drop(columns =['operating_system_version','model','manufacturer'], inplace=True)
devices_df = get_categ_col_dummy(devices_df)
devices_df.head()

,network_id,device_id,real_device,type_ip_camera,type_mobile_phone,type_pc,type_printer,type_tablet,type_voip,type_watch,operating_system_android,operating_system_axis firmware,operating_system_cisco ios,operating_system_cisco ios-xe,operating_system_cisco nx-os,operating_system_debian,operating_system_firefox os,operating_system_ios,operating_system_link-os,operating_system_linux,operating_system_mac os x,operating_system_miui os,operating_system_other,operating_system_symbian,operating_system_tizen,operating_system_watch os,operating_system_watchos,operating_system_windows
113,0,1028623,1028623.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
587,0,48047,48047.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
668,0,123568,123568.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
830,0,95366,95366.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
886,0,1755023,1755023.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [57]:
sessions_df = pd.read_csv(r'drive/My Drive/all_sessions.csv')
sessions_df['real_device'] = [float(str(id)+'.'+str(net)) for id,net in zip (sessions_df.device_id,sessions_df.network_id)]
sessions_df.drop(columns =['device_id'], inplace=True)
sessions_df.head()

,Unnamed: 0,network_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,real_device
0,0,0,1565074800,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,49152,TCP,790889.0,2.416641,4.339571,3.892985,0.028369,1.832509,NaN,1.602060,2.372912,1.414973,2.968950,2.668386,2.749261,NaN,2.281532,3.194237,0.000000,2.304026,NaN,2.724579,1.973128,1.973128,1.973128,1.973128,0.000000,2.496376,2.496376,2.496376,2.496376,0.0,35.0
1,1,0,1565053200,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,49153,TCP,1604622.0,2.252853,3.920489,4.457231,0.000000,1.230449,NaN,1.278754,1.949390,1.959041,2.670246,2.631444,2.666102,2.670246,1.144114,3.202216,3.202216,3.202216,3.202216,0.000000,2.032417,1.974972,1.981743,1.973128,0.725678,2.504335,2.504335,2.504335,2.503791,0.0,35.0
2,2,0,1565082000,e16257c983f2c35d41f39d425651972fa1905e46e968d7...,d43dad76e6cef2231d2efc743e498996b40f8b13fc120b...,443,TCP,NaN,1.832509,3.603577,0.000000,0.000000,1.612784,NaN,0.903090,1.832509,0.000000,3.546789,1.919078,2.759128,NaN,3.114217,0.000000,0.000000,0.000000,NaN,0.000000,1.813035,1.623249,1.655986,1.623249,0.986774,NaN,NaN,NaN,NaN,NaN,57.0
3,3,0,1565082000,1a4f860269acca6c264f00d84c4b63aad00b8f93a77250...,945e37dab8aee93dd4e650f8d17d76a3adfbc6aa70ebba...,443,TCP,NaN,1.662758,3.961848,0.000000,0.000000,0.602060,NaN,0.698970,1.662758,0.000000,3.531734,2.971276,3.359930,3.382377,3.008206,0.000000,0.000000,0.000000,0.000000,0.000000,2.357427,2.128953,2.292186,2.322219,1.631213,NaN,NaN,NaN,NaN,NaN,57.0
4,4,0,1565082000,df106cbe1ba4a700c00ec8883490f40a8afdb75c15a9ea...,4b43e85e630c2d18a0afaa2a6366367c4fc52d32b4ba5b...,443,TCP,NaN,1.000000,3.223236,0.000000,0.000000,0.000000,NaN,0.301030,1.000000,0.000000,3.223236,3.223236,3.223236,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,2.271067,2.271067,2.271067,2.271067,NaN,NaN,NaN,NaN,NaN,NaN,57.0


In [0]:
# url = 'https://armis-datahack.s3.amazonaws.com/all_sessions.csv'
# sessions_df = pd.read_csv(url)
# sessions_df['real_device'] = [float(str(id)+'.'+str(net)) for id,net in zip (sessions_df.device_id,sessions_df.network_id)]
# sessions_df.drop(columns =['device_id'], inplace=True)
# sessions_df.head()

In [0]:
#sessions_df = get_network_hierarchy(sessions_df)

In [0]:
numeric_col = ['packets_count',
       'outbound_bytes_count', 'inbound_bytes_count', 'packet_loss',
       'retransmit_count', 'latency', 'session_count',
       'outbound_packets_count', 'inbound_packets_count', 'outbound_bytes_max',
       'outbound_bytes_min', 'outbound_bytes_mean', 'outbound_bytes_median',
       'outbound_bytes_stddev', 'inbound_bytes_max', 'inbound_bytes_min',
       'inbound_bytes_mean', 'inbound_bytes_median', 'inbound_bytes_stddev',
       'outbound_packet_size_max', 'outbound_packet_size_min',
       'outbound_packet_size_mean', 'outbound_packet_size_median',
       'outbound_packet_size_stddev', 'inbound_packet_size_max',
       'inbound_packet_size_min', 'inbound_packet_size_mean',
       'inbound_packet_size_median', 'inbound_packet_size_stddev',]
for c in numeric_col:
  if sessions_df[c].dtype !='object':
    sessions_df[c] = np.log10(sessions_df[c]+1)
sessions_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log10
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app


,Unnamed: 0,network_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,real_device
0,0,0,1565074800,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,49152,TCP,790889.0,2.416641,4.339571,3.892985,0.028369,1.832509,NaN,1.602060,2.372912,1.414973,2.968950,2.668386,2.749261,NaN,2.281532,3.194237,0.000000,2.304026,NaN,2.724579,1.973128,1.973128,1.973128,1.973128,0.000000,2.496376,2.496376,2.496376,2.496376,0.0,35.0
1,1,0,1565053200,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,49153,TCP,1604622.0,2.252853,3.920489,4.457231,0.000000,1.230449,NaN,1.278754,1.949390,1.959041,2.670246,2.631444,2.666102,2.670246,1.144114,3.202216,3.202216,3.202216,3.202216,0.000000,2.032417,1.974972,1.981743,1.973128,0.725678,2.504335,2.504335,2.504335,2.503791,0.0,35.0
2,2,0,1565082000,e16257c983f2c35d41f39d425651972fa1905e46e968d7...,d43dad76e6cef2231d2efc743e498996b40f8b13fc120b...,443,TCP,NaN,1.832509,3.603577,0.000000,0.000000,1.612784,NaN,0.903090,1.832509,0.000000,3.546789,1.919078,2.759128,NaN,3.114217,0.000000,0.000000,0.000000,NaN,0.000000,1.813035,1.623249,1.655986,1.623249,0.986774,NaN,NaN,NaN,NaN,NaN,57.0
3,3,0,1565082000,1a4f860269acca6c264f00d84c4b63aad00b8f93a77250...,945e37dab8aee93dd4e650f8d17d76a3adfbc6aa70ebba...,443,TCP,NaN,1.662758,3.961848,0.000000,0.000000,0.602060,NaN,0.698970,1.662758,0.000000,3.531734,2.971276,3.359930,3.382377,3.008206,0.000000,0.000000,0.000000,0.000000,0.000000,2.357427,2.128953,2.292186,2.322219,1.631213,NaN,NaN,NaN,NaN,NaN,57.0
4,4,0,1565082000,df106cbe1ba4a700c00ec8883490f40a8afdb75c15a9ea...,4b43e85e630c2d18a0afaa2a6366367c4fc52d32b4ba5b...,443,TCP,NaN,1.000000,3.223236,0.000000,0.000000,0.000000,NaN,0.301030,1.000000,0.000000,3.223236,3.223236,3.223236,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,2.271067,2.271067,2.271067,2.271067,NaN,NaN,NaN,NaN,NaN,NaN,57.0


In [0]:
def range_list(val_list):
    min_val = min(val_list)
    max_val = max(val_list)

    return (max_val-min_val)
  
classic_functions = [np.nanmin,np.nanmax,np.nanmean,np.nanstd, np.nanmedian]

df_gb = sessions_df.groupby(['real_device'],as_index=False).agg({'host':'nunique',
                                                      'host_ip':'nunique',
                                                      'port_dst':'nunique',
                                                      'timestamp':['nunique','count',('range', lambda x: range_list(x))],
                                                      'transport_protocol':'nunique',
                                                      'packets_count':classic_functions,
                                                      'outbound_bytes_count':classic_functions, 
                                                      'inbound_bytes_count':classic_functions, 
                                                      'packet_loss':classic_functions,
                                                      'retransmit_count':classic_functions, 
                                                      'latency':classic_functions, 
                                                      'session_count':classic_functions,
                                                      'outbound_packets_count':classic_functions, 
                                                      'inbound_packets_count':classic_functions, 
                                                      'outbound_bytes_max':classic_functions,
                                                      'outbound_bytes_min':classic_functions, 
                                                      'outbound_bytes_mean':classic_functions, 
                                                      'outbound_bytes_median':classic_functions,
                                                      'outbound_bytes_stddev':classic_functions, 
                                                      'inbound_bytes_max':classic_functions, 
                                                      'inbound_bytes_min':classic_functions,
                                                      'inbound_bytes_mean':classic_functions, 
                                                      'inbound_bytes_median':classic_functions, 
                                                      'inbound_bytes_stddev':classic_functions,
                                                      'outbound_packet_size_max':classic_functions, 
                                                      'outbound_packet_size_min':classic_functions,
                                                      'outbound_packet_size_mean':classic_functions, 
                                                      'outbound_packet_size_median':classic_functions,
                                                      'outbound_packet_size_stddev':classic_functions, 
                                                      'inbound_packet_size_max':classic_functions,
                                                      'inbound_packet_size_min':classic_functions, 
                                                      'inbound_packet_size_mean':classic_functions,
                                                      'inbound_packet_size_median':classic_functions, 
                                                      'inbound_packet_size_stddev':classic_functions})

df_gb.columns = ["_".join(x) for x in df_gb.columns.ravel()]

In [0]:
final_df = pd.merge(devices_df, df_gb, how = 'right', right_on='real_device_' , left_on='real_device')
print(df_gb.shape, devices_df.shape,final_df.shape)

(76038, 153) (76261, 27) (76038, 180)


In [0]:
final_df = final_df.replace([np.inf, -np.inf], np.nan)
final_df = final_df.fillna(0)

In [0]:
# final_df.to_csv('features_v3.csv')

In [0]:
# from google.colab import files
# files.download("features_v3.csv")

In [0]:
from google.colab import drive
drive.mount('drive')
final_df.to_csv('features_v3.csv')
!cp features_v3.csv drive/My\ Drive/

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
